In [1]:
import sys
 
# setting path
sys.path.append('../')

from utils.display import display_pixel_arry

from dcm_anonymizers.img_anonymizers import DCMImageAnonymizer
# from dcm_anonymizers.phi_detectors import DcmRobustPHIDetector

from pydicom import dcmread

from pathlib import Path

In [2]:
validation_data_path = "/home/r079a/Desktop/de-identification/dataset/midi-validation-data"
sample_dicom = Path(validation_data_path, 'input_data/8172202354/3.4.823.0.1.5139398.5.009.3225903761765232034/3.4.823.0.1.5139398.5.009.2213188170240812441/00000105.dcm')

# sample_dicom = Path(validation_data_path, 'input_data/2951928526/2.1.846.0.0.0624952.3.426.1171423858789191618/2.1.846.0.0.0624952.3.426.4666769529569841014/00000001.dcm')

print(sample_dicom)

/home/r079a/Desktop/de-identification/dataset/midi-validation-data/input_data/8172202354/3.4.823.0.1.5139398.5.009.3225903761765232034/3.4.823.0.1.5139398.5.009.2213188170240812441/00000105.dcm


In [3]:
dataset = dcmread(sample_dicom)

In [4]:
pixel_data = dataset.get("pixel_array")

In [5]:
print("PixelData" in dataset)

True


In [6]:
# print(pixel_data[0].shape)

In [7]:
# display_pixel_arry(pixel_data[0])

In [8]:
# detector = DcmRobustPHIDetector()

In [9]:
# imganonymizer = DCMImageAnonymizer(phi_detector=detector)

In [10]:
# updated = imganonymizer.anonymize_dicom_image_data(dataset)

In [11]:
# updated

In [12]:
import pydicom
from pydicom.dataset import Dataset, DataElement
from pydicom.sequence import Sequence

from dcm_anonymizers.private_tags_extractor import PrivateTagsExtractor

In [13]:
ptags_dict = PrivateTagsExtractor('../docs/TCIAPrivateTagKB-02-01-2024-formatted.csv')

In [14]:
def extract_private_groups_n_creators(dataset):
    creators = []
    groups = []
    for element in dataset:
        if element.VR == 'OW':
            continue
        if element.tag.is_private:
            groups.append(f"{element.tag.group:04x}")
            if element.name == 'Private Creator' and element.value not in creators:                
                creators.append(element.value)

    groups = list(set(groups))

    return groups, creators

In [15]:
groups, creators = extract_private_groups_n_creators(dataset)

print(groups)
print(creators)

['0019', '0117', '0029', '0051', '0137']
['SIEMENS MR HEADER', 'SIEMENS CSA HEADER', 'SIEMENS MEDCOM HEADER2', 'UCSF BIRP PRIVATE CREATOR 011710XX', 'NetRAAD']


In [16]:
def create_pattents_from_element(element, parent_elements: list[DataElement]):
    patterns = []
    if len(parent_elements) == 0:
        patterns = ptags_dict.search_patterns_from_element(element, creators)
    else:
        element_pattens = ptags_dict.search_patterns_from_element(element, creators)
        for pttrn in element_pattens:
            filtered_rows = ptags_dict.filter_by_pattern_n_vr(pttrn, element.VR)
            print(pttrn, len(filtered_rows))                                                         
                                                              
        for pe in parent_elements:
            print(pe)

    return patterns

def traverse_dataset(dataset, parent_elements=[], is_root=True):
    for elem in dataset:
        tag = elem.tag
        VR = elem.VR
        value = elem.value
        name = elem.name

        # Check if the root element is private
        if is_root:
            if not elem.tag.is_private:
                continue
        
        # Process the element
        if isinstance(value, Sequence):
            # If the value is a Sequence, recursively traverse each Dataset in the Sequence
            # print(f"Sequence: {name} (Tag: {tag})")
            updated_parent_elements = parent_elements.copy()
            updated_parent_elements.append(elem)
            for i, item in enumerate(value):
                # print(f"  Item {i} (Parent: {name})")
                traverse_dataset(item, parent_elements=updated_parent_elements, is_root=False)
        else:
            # process the data element
            print(elem.tag)
            patterns = create_pattents_from_element(elem, parent_elements)
            print(patterns)
            print("------------------")

In [32]:
traverse_dataset(dataset)

(0019, 0010)
['\\(0019,SIEMENS MR HEADER.*,10\\)', '\\(0019,SIEMENS CSA HEADER.*,10\\)', '\\(0019,SIEMENS MEDCOM HEADER2.*,10\\)', '\\(0019,UCSF BIRP PRIVATE CREATOR 011710XX.*,10\\)', '\\(0019,NetRAAD.*,10\\)', '\\(0019,.*,10\\)']
------------------
(0019, 100b)
['\\(0019,SIEMENS MR HEADER.*,0b\\)', '\\(0019,SIEMENS CSA HEADER.*,0b\\)', '\\(0019,SIEMENS MEDCOM HEADER2.*,0b\\)', '\\(0019,UCSF BIRP PRIVATE CREATOR 011710XX.*,0b\\)', '\\(0019,NetRAAD.*,0b\\)', '\\(0019,.*,0b\\)']
------------------
(0019, 100c)
['\\(0019,SIEMENS MR HEADER.*,0c\\)', '\\(0019,SIEMENS CSA HEADER.*,0c\\)', '\\(0019,SIEMENS MEDCOM HEADER2.*,0c\\)', '\\(0019,UCSF BIRP PRIVATE CREATOR 011710XX.*,0c\\)', '\\(0019,NetRAAD.*,0c\\)', '\\(0019,.*,0c\\)']
------------------
(0019, 100d)
['\\(0019,SIEMENS MR HEADER.*,0d\\)', '\\(0019,SIEMENS CSA HEADER.*,0d\\)', '\\(0019,SIEMENS MEDCOM HEADER2.*,0d\\)', '\\(0019,UCSF BIRP PRIVATE CREATOR 011710XX.*,0d\\)', '\\(0019,NetRAAD.*,0d\\)', '\\(0019,.*,0d\\)']
---------------